In [55]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("white")

import time
import timeit

import scipy.stats 
import pandas as pd
import pymc as pm

import re
import numpy as np
from datetime import datetime

In [56]:
dateconv = lambda x: datetime.strptime(x, "%Y%m%d %H:%M:%S")

print "Date: ", dateconv("20160114  17:00:00")

Date:  2016-01-14 17:00:00


In [57]:
import os
minuteDirectory = "../data/unzipped_min/"
minuteFiles = np.array(os.listdir(minuteDirectory))
print "Directory: ", minuteFiles[0:5]

Directory:  ['ZS201405_20140421.csv' 'ZS201405_20140422.csv' 'ZS201405_20140423.csv'
 'ZS201405_20140424.csv' 'ZS201405_20140425.csv']


In [58]:
names=("date","open","high","low","close","volume","adjclose")
types=("|S18", float, float, float, float, int, float)

def getFullFileName(minuteDirectory, minuteFile):
    return minuteDirectory + "/" + minuteFile
    

def readMinuteFile(minuteFileName):
    data = np.genfromtxt(minuteFileName, delimiter=",", skip_header=1, names=names, dtype=None, converters={'date': dateconv})
    #data = np.genfromtxt(minuteFileName, delimiter=",", skip_header=1, names=names, dtype=None)
    return data

m_data = readMinuteFile(getFullFileName(minuteDirectory, minuteFiles[0]))
print "m_data: ", m_data[0]
print "m_data[date]: ", m_data["date"][0:5]

#print "m_data: ", m_data
#print "m_data: ", np.asarray(m_data)



m_data:  (datetime.datetime(2014, 4, 20, 17, 0), 1515.0, 1518.5, 1514.25, 1517.5, 233, 1516.1)
m_data[date]:  [datetime.datetime(2014, 4, 20, 17, 0)
 datetime.datetime(2014, 4, 20, 17, 1)
 datetime.datetime(2014, 4, 20, 17, 2)
 datetime.datetime(2014, 4, 20, 17, 3)
 datetime.datetime(2014, 4, 20, 17, 4)]


In [60]:
t = timedelta(minutes = 10)
print "T: ", t

T:  0:10:00


In [76]:
def getFileDate(fileName):
    rDate = fileName.replace('.', '_').split('_')[1]
    return datetime.strptime(rDate, "%Y%m%d")
v_getFileDate = np.vectorize(getFileDate)

def getFilesForPeriod(minuteFiles, start, end):
    fileDates = v_getFileDate(minuteFiles)
    return fileDates[(fileDates >= start) & (fileDates <= end)]

startDate = datetime(2014,4,21,0,0,0)
endDate =  start + timedelta(hours=6)
print "Files for Period ", startDate, " - ", endDate, " -- ", getFilesForPeriod(minuteFiles, startDate, endDate)

Files for Period  2014-04-21 00:00:00  -  2014-04-21 06:00:00  --  [datetime.datetime(2014, 4, 21, 0, 0)]


In [61]:
def getDataForPeriod(m_data, lastMinuteProcessed, durationMinutes):
    nextIntervalEnd = lastMinuteProcessed + timedelta(minutes = durationMinutes)
    withinWindow = lambda x: x["date"] > lastMinuteProcessed and x["date"] < nextIntervalEnd
    return m_data[(m_data["date"] > lastMinuteProcessed) & (m_data["date"] <= nextIntervalEnd)]

lastMinuteProcessed = m_data["date"][0] - timedelta(seconds=1)
filteredData = getDataForPeriod(m_data, lastMinuteProcessed, 10)
print "Last Minute Processed: ", lastMinuteProcessed
print "Filtered Data:\n", filteredData

Last Minute Processed:  2014-04-20 16:59:59
Filtered Data:
[ (datetime.datetime(2014, 4, 20, 17, 0), 1515.0, 1518.5, 1514.25, 1517.5, 233, 1516.1)
 (datetime.datetime(2014, 4, 20, 17, 1), 1517.25, 1518.25, 1517.0, 1517.0, 46, 1517.6)
 (datetime.datetime(2014, 4, 20, 17, 2), 1516.75, 1518.0, 1516.5, 1517.5, 33, 1517.425)
 (datetime.datetime(2014, 4, 20, 17, 3), 1517.5, 1518.25, 1517.5, 1518.0, 22, 1517.975)
 (datetime.datetime(2014, 4, 20, 17, 4), 1518.0, 1519.5, 1518.0, 1519.25, 106, 1518.925)
 (datetime.datetime(2014, 4, 20, 17, 5), 1520.0, 1521.0, 1519.5, 1519.75, 77, 1520.475)
 (datetime.datetime(2014, 4, 20, 17, 6), 1520.0, 1520.5, 1519.75, 1519.75, 12, 1520.2)
 (datetime.datetime(2014, 4, 20, 17, 7), 1520.0, 1521.5, 1520.0, 1521.0, 66, 1520.975)
 (datetime.datetime(2014, 4, 20, 17, 8), 1521.25, 1522.75, 1521.25, 1522.75, 33, 1522.25)
 (datetime.datetime(2014, 4, 20, 17, 9), 1522.75, 1523.75, 1522.5, 1523.5, 93, 1523.3)]


In [47]:
def getMeanValuation(m_data):
    return (np.mean(m_data["open"]), np.mean(m_data["close"]), np.mean(np.absolute(m_data["high"] - m_data["low"])))
print "m_data: ", np.concatenate((m_data["close"], m_data["open"]), axis=0)

m_open, m_close, m_diff = getMeanValuation(m_data)
print "m_open:  ", m_open
print "m_close: ", m_close
print "m_diff:  ", m_diff

 m_data:  [ 896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25
  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25
  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25
  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25
  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25
  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25
  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25
  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25
  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25
  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25
  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25
  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25
  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25  896.25
  896.25  896.25  896.25  896.25  896.25 